In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-cis-12nd-solution-part-1/__output__.json
/kaggle/input/ieee-cis-12nd-solution-part-1/X_test2.pkl
/kaggle/input/ieee-cis-12nd-solution-part-1/X_train2.pkl
/kaggle/input/ieee-cis-12nd-solution-part-1/__results__.html
/kaggle/input/ieee-cis-12nd-solution-part-1/__notebook__.ipynb
/kaggle/input/ieee-cis-12nd-solution-part-1/y_train2.pkl
/kaggle/input/ieee-cis-12nd-solution-part-1/custom.css


In [2]:
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM
!apt-get install -y -qq libboost-all-dev

Cloning into 'LightGBM'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 14696 (delta 11), reused 3 (delta 0), pack-reused 14659
Receiving objects: 100% (14696/14696), 10.22 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (10553/10553), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/kaggle/working/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 5.06 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package autotools-dev.
(Reading database ... 60469 files and directories cur

In [3]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 6.3.0
-- The CXX compiler identification is GNU 6.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Try OpenMP C flag = [-fopenmp]
-- Performing Test OpenMP_FLAG_DETECTED
-- Performing Test OpenMP_FLAG_DETECTED - Success
-- Try OpenMP CXX flag = [-fopenmp]
-- Performing Test OpenMP_FLAG_DETECTED
-- Performing Test OpenMP_FLAG_DETECTED - Success
-- Found OpenMP: -fopenmp  
-- Looking for CL_VERSION_2_0
-- Looking for CL_VERSION_2_0 - not found
-- Looking for CL_VERSION_1_2
-- Looking for CL_VERSION_

rm: cannot remove 'build': No such file or directory


In [4]:
!cd LightGBM/python-package/;python3 setup.py install --precompile

running install
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
copying lightgbm/VERSION.t

In [5]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
from sklearn.metrics import roc_auc_score
import sqlite3
import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
import hashlib

In [7]:
import os
__print__ = print
def print(string):
    __print__(string)
    os.system(f'echo \"{string}\"')

In [8]:
X_train = pd.read_pickle('../input/ieee-cis-12nd-solution-part-1/X_train2.pkl')
X_test = pd.read_pickle('../input/ieee-cis-12nd-solution-part-1/X_test2.pkl')
y_train = pd.read_pickle('../input/ieee-cis-12nd-solution-part-1/y_train2.pkl')
y_train = y_train.isFraud

In [9]:
cat = ['uid1','id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9','hour','dow','device_name', 'OS_id_30',  'browser_id_31','ProductID','DeviceInfo__P_emaildomain', 
        'card1__card5', 
        'card2__id_20',
        'card5__P_emaildomain', 
        'addr1__card1',
        'addr1__addr2',
        'card1__card2',
        'card2__addr1',
        'card1__P_emaildomain',
        'card2__P_emaildomain',
        'addr1__P_emaildomain',
        'DeviceInfo__id_31',
        'DeviceInfo__id_20',
        'DeviceType__id_31',
        'DeviceType__id_20',
        'DeviceType__P_emaildomain',
        'card1__M4',
        'card2__M4',
        'addr1__M4',
        'P_emaildomain__M4',
       'uid1__ProductID',
       'uid1__DeviceInfo']

In [10]:
cat = set(cat) & set(X_train.columns)

In [11]:

for column in cat:
    train_set = set(X_train[column])
    test_set = set(X_test[column])
    tt = train_set.intersection(test_set)
    print('----------------------------------------')
    print(column)
    print(f'train:{len(tt)/len(train_set)}')
    print(f'test:{len(tt)/len(test_set)}')
    X_train[column] = X_train[column].map(lambda x: -999 if x not in tt else x)
    X_test[column] = X_test[column].map(lambda x: -999 if x not in tt else x)

----------------------------------------
card1__card5
train:0.6877796901893287
test:0.699922931408954
----------------------------------------
uid1
train:0.12073629998472034
test:0.13567933094626056
----------------------------------------
addr1__P_emaildomain
train:0.7479224376731302
test:0.7792207792207793
----------------------------------------
R_emaildomain
train:1.0
test:1.0
----------------------------------------
id_31
train:0.7175572519083969
test:0.6911764705882353
----------------------------------------
card1__card2
train:0.6867254199944919
test:0.700765825897562
----------------------------------------
card1__P_emaildomain
train:0.5412019042800303
test:0.5949936788874842
----------------------------------------
addr1__M4
train:0.6373983739837399
test:0.6962699822380106
----------------------------------------
uid1__DeviceInfo
train:0.1028989375475554
test:0.1150085488352212
----------------------------------------
addr1__addr2
train:0.5136986301369864
test:0.56962025316455

In [12]:
# X_train = X_train[:10000]
# y_train = y_train[:10000]

In [13]:
kf=KFold(n_splits = 5)
resu1 = 0
impor1 = 0
y_pred = 0
stack_train = np.zeros([X_train.shape[0],])
for train_index, test_index in kf.split(X_train, y_train):
    X_train2= X_train.iloc[train_index,:]
    y_train2= y_train.iloc[train_index]
    X_test2= X_train.iloc[test_index,:]
    y_test2= y_train.iloc[test_index]
    clf = lgb.LGBMClassifier(n_estimators=100000, random_state=42,subsample=0.7,device="gpu",
                             colsample_bytree=0.7,learning_rate=0.005,importance_type = 'gain', #lr记得改
                     max_depth = -1, num_leaves = 256,min_child_samples=20,min_split_gain = 0.001,
                       bagging_freq=1,reg_alpha = 0,reg_lambda = 0,n_jobs = -1,metric='None')
    clf.fit(X_train2,y_train2,eval_set = [(X_train2,y_train2),(X_test2,y_test2)], eval_metric = 'auc',early_stopping_rounds=500,verbose=100) #early_stopping_rounds记得改
    temp_predict = clf.predict_proba(X_test2)[:,1]
    stack_train[test_index] = temp_predict
    y_pred += clf.predict_proba(X_test)[:,1]/5
    roc = roc_auc_score(y_test2, temp_predict)
    print(roc)
    resu1 += roc/5
    impor1 += clf.feature_importances_/5
    gc.collect()
print(f'End:{resu1}')

Training until validation scores don't improve for 500 rounds
[100]	training's auc: 0.94843	valid_1's auc: 0.896977
[200]	training's auc: 0.965301	valid_1's auc: 0.907935
[300]	training's auc: 0.975576	valid_1's auc: 0.913588
[400]	training's auc: 0.982497	valid_1's auc: 0.918373
[500]	training's auc: 0.987842	valid_1's auc: 0.921624
[600]	training's auc: 0.991633	valid_1's auc: 0.925254
[700]	training's auc: 0.994416	valid_1's auc: 0.928085
[800]	training's auc: 0.996248	valid_1's auc: 0.930461
[900]	training's auc: 0.997499	valid_1's auc: 0.932513
[1000]	training's auc: 0.998381	valid_1's auc: 0.934241
[1100]	training's auc: 0.998931	valid_1's auc: 0.935883
[1200]	training's auc: 0.9993	valid_1's auc: 0.937004
[1300]	training's auc: 0.999532	valid_1's auc: 0.937976
[1400]	training's auc: 0.999683	valid_1's auc: 0.938759
[1500]	training's auc: 0.999782	valid_1's auc: 0.939644
[1600]	training's auc: 0.999854	valid_1's auc: 0.940289
[1700]	training's auc: 0.999902	valid_1's auc: 0.94077

In [14]:
resu = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
resu['isFraud'] = y_pred
resu.to_csv('lgb.csv',index=False)
a= pd.DataFrame()
a['train'] = stack_train
a.to_csv('lgb_train.csv',index=False)